In [26]:
# basic imports from brightway
import bw2analyzer as ba
import bw2calc as bc
import bw2data as bd
from bw2data import databases
import bw2io as bi
from bw2io import ExcelImporter
from bw2io.importers import SingleOutputEcospold2Importer
import bw2analyzer as bwa
from bw2data import methods
import argparse
import bw2data as bd

# other relevant packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path

In [27]:
# define a project where we install the databases and work in this script
bd.projects.set_current('brightway25')

In [28]:
bd.databases

Databases dictionary with 4 object(s):
	biosphere3
	bw25_db
	ecoinvent-3.10-biosphere
	ecoinvent310clca

In [29]:
ei_clca = bd.Database('ecoinvent310clca')
ei_bio = bd.Database('ecoinvent-3.10-biosphere')
test = bd.Database('bw25_db')
el_bio3 = bd.Database('biosphere3')

In [30]:
excel_db = bi.ExcelImporter("C:\\Users\\TimWeber\\OneDrive - 2.-0 LCA Consultants ApS\\Intranet - Brightway2\\excel_model_example\\brightway25_excel_db_template.xlsx")

Extracted 1 worksheets in 0.03 seconds


In [31]:
excel_db.apply_strategies()

Applying strategy: csv_restore_tuples
Applying strategy: csv_restore_booleans
Applying strategy: csv_numerize
Applying strategy: csv_drop_unknown
Applying strategy: csv_add_missing_exchanges_section
Applying strategy: normalize_units
Applying strategy: normalize_biosphere_categories
Applying strategy: normalize_biosphere_names
Applying strategy: strip_biosphere_exc_locations
Applying strategy: set_code_by_activity_hash
Applying strategy: link_iterable_by_fields
Applying strategy: assign_only_product_as_production
Applying strategy: link_technosphere_by_activity_hash
Applying strategy: drop_falsey_uncertainty_fields_but_keep_zeros
Applying strategy: convert_uncertainty_types_to_integers
Applying strategy: convert_activity_parameters_to_list
Applied 16 strategies in 0.25 seconds


In [32]:
excel_db.match_database(fields=['name','location'])

Applying strategy: link_iterable_by_fields


In [33]:
excel_db.match_database(db_name='ecoinvent310clca',
                        fields=['name','location','unit'])

Applying strategy: link_iterable_by_fields


In [34]:
excel_db.match_database(db_name='ecoinvent-3.10-biosphere',
                        fields=['name','categories','unit'])

Applying strategy: link_iterable_by_fields


In [35]:
# check foreground imports for unlinked processes
pd.DataFrame(excel_db.unlinked)

""


In [36]:
excel_db.write_database()

100%|██████████| 1/1 [00:00<00:00, 1721.09it/s]


14:44:51+0200 [info     ] Vacuuming database            
Created database: bw25_db


In [38]:
data_ceramics = [activity.as_dict() for activity in test]


In [39]:
df_ceramics = pd.DataFrame(data_ceramics)


In [40]:
from IPython.display import display
display(df_ceramics)

,production amount,unit,location,code,reference product,comment,name,worksheet name,database,type,id
0,1,kilogram,GLO,ceramics,ceramics,None,ceramics production,LCI,bw25_db,processwithreferenceproduct,214726098899460096


In [41]:
# Get all activities and products
activities = set(test)  # All activities in the database

# Get all products linked via production exchanges
products = set([
    exc.input for act in test
    for exc in act.exchanges() if exc.input == act.key
])

# Identify mismatches
extra_activities = activities - products
extra_products = products - activities

print("Extra activities (no matching products):", extra_activities)
print("Extra products (no matching activities):", extra_products)

Extra activities (no matching products): set()
Extra products (no matching activities): set()


In [46]:
ceramics_new = test.get(name='ceramics production')

In [47]:
lcia_gwp100 = ('EF v3.1', 'climate change', 'global warming potential (GWP100)')
lcia_water = ('EF v3.1','water use','user deprivation potential (deprivation-weighted water consumption)')
lcia_land = ('EF v3.1', 'land use', 'soil quality index')

In [48]:
ceramics_lca = ceramics_new.lca(lcia_gwp100)
ceramics_lca.score


0.15302178136694788